In [3]:
import string
import pandas as pd
import sparql_dataframe
from rdflib import Graph

In [4]:
sparql_prefixes = """
        prefix crypto: <http://www.w3.org/2000/10/swap/crypto#>
        prefix dcterms: <http://purl.org/dc/terms/>
        prefix fno: <https://w3id.org/function/ontology#>
        prefix list: <http://www.w3.org/2000/10/swap/list#>
        prefix log: <http://www.w3.org/2000/10/swap/log#>
        prefix math: <http://www.w3.org/2000/10/swap/math#>
        prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        prefix string: <http://www.w3.org/2000/10/swap/string#>
        prefix time: <http://www.w3.org/2000/10/swap/time#>
        prefix xsd: <http://www.w3.org/2001/XMLSchema#>
"""

In [5]:
g = Graph()
g.parse('fno-builtins.n3', format='n3')
dic_prefixes = {
    "http://www.w3.org/2000/10/swap/crypto": 'crypto' ,
    "http://www.w3.org/2000/10/swap/list": 'list' ,
    "http://www.w3.org/2000/10/swap/log": 'log' ,
    "http://www.w3.org/2000/10/swap/math": 'math' ,
    "http://www.w3.org/2000/10/swap/string": 'string' ,
    "http://www.w3.org/2000/10/swap/time": "time" ,
}



In [6]:
query = sparql_prefixes + """
    
    select distinct (strbefore(str(?s), '#') as ?namespace) 
    where { ?s a fno:Function . }
    order by ?namespace
"""
prefixes = [dic_prefixes[str(row.namespace)] for row in g.query(query)]

In [7]:
md_string = ""

In [8]:
for p in prefixes:
    md_string += f"## {p} ##" + " {#" + p + "}\n"
    q = sparql_prefixes + """
        select (strafter(str(?s), '#') as ?name) ?s ?comment ?exampleDescription ?example
        where {
            ?s a fno:Function ;
                fno:name ?name ;
                dcterms:comment ?comment ;
            .
            filter(strstarts(str(?s), "$NAMESPACE"))            
        }
    """
    # name and description
    query = string.Template(q).substitute(NAMESPACE=next(key for key, value in dic_prefixes.items() if value == p))
    for func in g.query(query):
        md_string += "### " + p + ":" + func.name + "() ### {#" + str(func.name) + "}\n"
        md_string += func.comment + "\n\n"

        # examples        
        query = sparql_prefixes + """
            select ?description ?seeAlso ?expression
            where {
                <$FUNCTION> a fno:Function ;
                    fno:example ( [ a fno:Test ;
                        dcterms:description ?description ;
                        rdfs:seeAlso ?seeAlso ;
                        fno:expression ?expression ]) .
            }
            """
        query = string.Template(query).substitute(FUNCTION=str(func.s))
        for example in g.query(query):
            e = """
<div class=example>
$DESCRIPTION
```$EXPRESSION```
</div>            
"""
            expression = string.Template(e).substitute(DESCRIPTION=str(example.description), EXPRESSION=str(example.expression))
            md_string += "[Try in Notation3 Editor &#128640;](" + str(example.seeAlso) + ")\n\n"            
            md_string += expression + "\n"
            

In [9]:
print(md_string)

## crypto ## {#crypto}
### crypto:sha() ### {#sha}
The object is a SHA-1 hash of the subject.

[Try in Notation3 Editor &#128640;](http://ppr.cs.dal.ca:3002/n3/editor/?formula=%40prefix%20%3A%20%3C%3A%3E%20.%0A%40prefix%20string%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F10%2Fswap%2Fstring%23%3E%20.%0A%0A%3ALet%20%3Astring%20%22placeholder%22%20.%0A%0A%7B%20%3ALet%20%3Astring%20%3Fx%20.%20%7D%0A%3D%3E%0A%7B%20%3Aresult%20%3Ais%20%3Fx%20.%20%7D.%20)


<div class=example>

This is an example for a built-in function.
            
```
@prefix : <:> .
@prefix string: <http://www.w3.org/2000/10/swap/string#> .
:Let :string "placeholder" .
{ :Let :string ?x . } => { :result :is ?x . } .
            ```
</div>            

## list ## {#list}
### list:append() ### {#append}
Concatenates separate lists into a single list. 
`true` if and only if the subject is a list of lists and the concatenation of all those lists is `$o`.
`$o` can be calculated as a function of the subject. 
Example: `( (1 2) (3